# TRATAMIENTO DE DATOS CON PYSPARK

In [170]:
# 1. CONFIGURACIÓN E IMPORTACIONES

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

# Iniciamos Spark
spark = SparkSession.builder \
    .appName("FinPlus_ETL_Limpieza") \
    .getOrCreate()

# Rutas de los datos
DATA_PATH = "/home/jovyan/work/data/"  
OUTPUT_PATH = "/home/jovyan/work/data/curated/"

In [171]:
# 2. INGESTA DE DATOS 

# Cargar CLIENTS.csv
# Usamos header=True para leer la cabecera e inferSchema=True para detectar números
df_clients = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# Cargar BEHAVIOURAL.parquet
df_behav = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")

# Verificamos qué columnas tenemos 
print("Esquema Clients:")
df_clients.printSchema()

print("Esquema Behavioural:")
df_behav.printSchema()

Esquema Clients:
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = true)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = true)
 |-- FAMILY_SIZE: double (nullable = true)
 |-- REACTIVE_SCORING: double (nullable = true)
 |-- PROACTIVE_SCORING: double (nullable = true)
 |-- BEHAVIORAL_SCORING: double (nullable = true)
 |-- DAYS_LAST_INFO_CHANGE: d

Primero analizamos las variables de tipo string. Hay de distintos tipos:

- Categóricas puras: "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY" (en ambas tablas).

- Categóricas numéricas: "NON_COMPLIANT_CONTRACT", "DIGITAL_CLIENT".

- Fecha: "DATE" (sólo en behavioural).

- Identificadores: "CLIENT_ID" (en ambas tablas), "CONTRACT_ID".

Analizamos las filas duplicadas

In [172]:
def auditar_duplicados_completo(df, nombre_tabla):
    print(f"VERIFICACIÓN DE DUPLICADOS: {nombre_tabla}\n ")
    
    # 1. Cálculos Básicos
    total_rows = df.count()
    distinct_rows = df.distinct().count()
    num_duplicados = total_rows - distinct_rows
    
    print(f"• Total filas:      {total_rows}")
    print(f"• Filas únicas:     {distinct_rows}")
    print(f"• Duplicados:       {num_duplicados}")
    
    # 2. Lógica Condicional
    if num_duplicados > 0:
        pct = (num_duplicados / total_rows) * 100
        print(f"\n AVISO: Hay {num_duplicados} filas repetidas ({pct:.2f}%).")
        print("   Mostrando ejemplos de filas idénticas:")
        
        # Esta parte solo se ejecuta si hay duplicados 
        # Agrupamos por TODAS las columnas para encontrar filas 100% idénticas
        (df.groupBy(df.columns)
           .count()
           .where(F.col("count") > 1)
           .orderBy(F.col("count").desc()) # Ponemos las más repetidas arriba
           .show(5, truncate=False))
        
    else:
        print("\nLimpio. No existen filas duplicadas exactas.")
        

# --- EJECUCIÓN ---
auditar_duplicados_completo(df_clients, "CLIENTS")
auditar_duplicados_completo(df_behav, "BEHAVIOURAL")

VERIFICACIÓN DE DUPLICADOS: CLIENTS
 
• Total filas:      162977
• Filas únicas:     162977
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.
VERIFICACIÓN DE DUPLICADOS: BEHAVIOURAL
 
• Total filas:      1724854
• Filas únicas:     1724854
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.


Ahora analizamos los posibles IDs duplicados

In [173]:
def auditar_clave_primaria(df, col_id, nombre_tabla):
    print(f"VERIFICACIÓN DE CLAVE ÚNICA ({col_id}): {nombre_tabla}\n ")
    
    # 1. Contamos filas totales
    total = df.count()
    
    # 2. Contamos IDs únicos
    unicos = df.select(col_id).distinct().count()
    
    # 3. Diferencia
    dif = total - unicos
    
    if dif > 0:
        print(f"AVISO: Hay {dif} IDs repetidos que NO son filas idénticas.")
        print("   Esto significa que tienes clientes con datos conflictivos.")
        print("   Ejemplo de IDs repetidos:")
        
        # Mostramos cuáles son los culpables
        (df.groupBy(col_id)
           .count()
           .where(F.col("count") > 1)
           .show(5))
    else:
        print(f"CORRECTO: La columna {col_id} es una clave primaria única.")

# Ejecutamos solo para CLIENTS
auditar_clave_primaria(df_clients, "CLIENT_ID", "CLIENTS")

VERIFICACIÓN DE CLAVE ÚNICA (CLIENT_ID): CLIENTS
 
CORRECTO: La columna CLIENT_ID es una clave primaria única.


Como vemos qu eno hay duplicados, no es necesario hacer limpieza de estos. Nos enfocaremos en los NaNs.

In [174]:
# 1. Definimos la función mejorada (le añadimos un título para que quede claro)
def inspeccionar_datos(df, nombre_tabla):
    
    print(f"INSPECCIÓN DE: {nombre_tabla}\n")
    
    print(f"Dimensiones: {df.count()} filas x {len(df.columns)} columnas")
    
    print(f"\n--- 1. Conteo de Nulos ---")
    # Calculamos nulos
    exprs_nulos = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]
    # Mostramos verticalmente para leer mejor
    df.agg(*exprs_nulos).show(vertical=True, truncate=False)

# 2. Ejecutamos la inspección (SIN meterlo dentro de otro print)
inspeccionar_datos(df_clients, "CLIENTS")
inspeccionar_datos(df_behav, "BEHAVIOURAL")

INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0-----------------------------
 CLIENT_ID                   | 0      
 NON_COMPLIANT_CONTRACT      | 0      
 NAME_PRODUCT_TYPE           | 0      
 GENDER                      | 0      
 TOTAL_INCOME                | 0      
 AMOUNT_PRODUCT              | 0      
 INSTALLMENT                 | 7      
 EDUCATION                   | 39640  
 MARITAL_STATUS              | 2      
 HOME_SITUATION              | 0      
 REGION_SCORE                | 0      
 AGE_IN_YEARS                | 0      
 JOB_SENIORITY               | 29174  
 HOME_SENIORITY              | 0      
 LAST_UPDATE                 | 0      
 OWN_INSURANCE_CAR           | 0      
 CAR_AGE                     | 107550 
 FAMILY_SIZE                 | 2      
 REACTIVE_SCORING            | 91901  
 PROACTIVE_SCORING           | 337    
 BEHAVIORAL_SCORING          | 32246  
 DAYS_LAST_INFO_CHANGE       | 1      
 NUMBER_OF_

Se observa que solamente tendremos qu etratar nulls de la tabla de clientes. Pero además, no nos interesa eliminar filas porque eliminaríamos clientes, y menos en variables con tantos nulls como CAR_AGE, donde estaríamos eliminando más del 66% de los clientes. La estrategia ganadora en Big Data es "Imputar lo masivo, borrar lo anecdótico".

Primero de todo, eliminaremos a los clientes qu eno proporcionen casi datos, pues prácticamente es como si no existiese.

In [175]:
# ELIMINAR "CLIENTES ZOMBIE" (Filas con demasiados nulos)

print(f"Filas antes de limpieza fina: {df_clients.count()}")

# Tenemos unas 45 columnas. Si a un cliente le faltan más de 20 datos, no nos sirve.
# thresh=25 significa: "Mantener solo si tiene al menos 25 columnas con datos válidos"
df_clients = df_clients.dropna(thresh=25) 

print(f"Filas tras limpieza fina: {df_clients.count()}")

Filas antes de limpieza fina: 162977
Filas tras limpieza fina: 161078


In [176]:

# 3. LIMPIEZA Y TRANSFORMACIÓN

# A) LIMPIEZA DE STRINGS (CLIENTS)
# Lista de tus columnas categóricas reales (copiadas de tu esquema)
cols_categ_puras = [
    "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY"
]

# Normalizamos: quitamos espacios (trim) y pasamos a mayúsculas o minúsculas
for col_name in cols_categ_puras:
    # Solo si la columna existe en el dataframe
    if col_name in df_clients.columns:
        df_clients = df_clients.withColumn(col_name, F.trim(F.upper(F.col(col_name))))

# B) CONVERSIÓN DE FECHAS (BEHAVIOURAL)
# Tu columna DATE es string, hay que pasarla a formato fecha
# Spark suele ser listo, pero si falla, prueba con formato específico ej: "dd/MM/yyyy"
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))



In [177]:
# C) LIMPIEZA DE NULOS 

# 1. GRUPO "NO APLICA" (Rellenar con -1)
# Variables donde Nulo significa "No tiene" o "No disponible"
# CAR_AGE (66% nulos), JOB_SENIORITY, SCORING...
cols_flag = ["CAR_AGE", "JOB_SENIORITY", "REACTIVE_SCORING", "BEHAVIORAL_SCORING", "PROACTIVE_SCORING"]
df_clients = df_clients.fillna(-1, subset=cols_flag)

# 2. GRUPO "SIN HISTORIAL" (Rellenar con 0)
# El grupo de los 8770 nulos. Si no hay datos de préstamos, asumimos 0.
# Buscamos todas las columnas de préstamos (LOAN_) y estados (NUM_STATUS_)
cols_financieras = [c for c in df_clients.columns if c.startswith("LOAN_") or c.startswith("NUM_")]
# Añadimos otras que tengan sentido ser 0
cols_financieras.extend(["NUM_PREVIOUS_LOAN_APP", "NUMBER_OF_PRODUCTS", "Num_flag_insured"]) # Asegúrate de usar el nombre exacto (mayusc/minusc)

# Filtramos solo las que existen en el DF para no dar error
cols_financieras = [c for c in cols_financieras if c in df_clients.columns]
df_clients = df_clients.fillna(0, subset=cols_financieras)

# 3. GRUPO "CATEGÓRICO DESCONOCIDO" (Rellenar con 'Unknown')
# EDUCATION tiene 39k nulos. No podemos inventárnosla.
cols_categ_nulos = ["EDUCATION", "EMPLOYER_ORGANIZATION_TYPE"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_categ_nulos)

# 4. GRUPO "ANECDÓTICO" (Estrategia: Salvar al Cliente)
# Al ser poquísimos nulos, preferimos imputar para no perder la ficha del cliente.

# A) Para Numéricas (Installment): Usamos la MEDIANA (más robusta que la media)
# Calculamos la mediana aproximada (approxQuantile es muy eficiente en Spark)
mediana_inst = df_clients.stat.approxQuantile("INSTALLMENT", [0.5], 0.01)[0]
df_clients = df_clients.fillna(mediana_inst, subset=["INSTALLMENT"])

# B) Para Categóricas (Marital Status, Family Size): Usamos "UNKNOWN"
# Usar la moda requeriría más código, y para 2 filas no merece la pena el coste computacional.
# "UNKNOWN" nos permite ver en el Dashboard si hay errores de calidad.
cols_anecdoticas = ["MARITAL_STATUS", "FAMILY_SIZE"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_anecdoticas)

print("Limpieza completada.")

Limpieza completada.


In [178]:
inspeccionar_datos(df_clients, "CLIENTS")


INSPECCIÓN DE: CLIENTS

Dimensiones: 161078 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0--------------------------
 CLIENT_ID                   | 0   
 NON_COMPLIANT_CONTRACT      | 0   
 NAME_PRODUCT_TYPE           | 0   
 GENDER                      | 0   
 TOTAL_INCOME                | 0   
 AMOUNT_PRODUCT              | 0   
 INSTALLMENT                 | 0   
 EDUCATION                   | 0   
 MARITAL_STATUS              | 0   
 HOME_SITUATION              | 0   
 REGION_SCORE                | 0   
 AGE_IN_YEARS                | 0   
 JOB_SENIORITY               | 0   
 HOME_SENIORITY              | 0   
 LAST_UPDATE                 | 0   
 OWN_INSURANCE_CAR           | 0   
 CAR_AGE                     | 0   
 FAMILY_SIZE                 | 0   
 REACTIVE_SCORING            | 0   
 PROACTIVE_SCORING           | 0   
 BEHAVIORAL_SCORING          | 0   
 DAYS_LAST_INFO_CHANGE       | 0   
 NUMBER_OF_PRODUCTS          | 0   
 OCCUPATION                  | 0   
 DIGITAL

In [ ]:

# 5. INTEGRACIÓN (JOIN) - MODO NOMBRES ORIGINALES


print(" FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)\n")

col_id = "CLIENT_ID" 

# --- A. SNAPSHOT (Último dato por cliente) ---
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))

# Ventana para quedarnos con el último registro
w = Window.partitionBy(col_id).orderBy(F.desc("DATE"))
df_behav_dedup = df_behav.withColumn("rank", F.row_number().over(w)) \
                         .filter(F.col("rank") == 1) \
                         .drop("rank")

# --- B. JOIN ---
print(f"Uniendo tablas por {col_id}...")
df_master = df_clients.join(df_behav_dedup, on=col_id, how="left")

# --- C. INTEGRIDAD ---
filas_clientes = df_clients.count()
filas_master = df_master.count()

if filas_master > filas_clientes:
    diff = filas_master - filas_clientes
    print(f"AVISO: Aún hay {diff} duplicados.")
else:
    print(f"CORRECTO: 1 Cliente = 1 Fila.")

# --- D. LIMPIEZA POST-JOIN (Nulos a 0) ---
# Tras el join, las columnas financieras de BEHAVIOURAL pueden tener nulos
cols_financieras = [
    "CREDICT_CARD_BALANCE", "CREDIT_CARD_LIMIT", "NUMBER_DRAWINGS", 
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", 
    "CREDIT_CARD_DRAWINGS_OTHER", "CREDIT_CARD_DRAWINGS", "CREDIT_CARD_PAYMENT"
]

cols_existentes = [c for c in cols_financieras if c in df_master.columns]
df_master = df_master.fillna(0, subset=cols_existentes)

# E. INFORMACIÓN DE LA TABLA 
print(" ESTADO ACTUAL (PRE-REDUCCIÓN)\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print("\nEsquema Actual ")
df_master.printSchema()

 FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)

Uniendo tablas por CLIENT_ID...
CORRECTO: 1 Cliente = 1 Fila.
 ESTADO ACTUAL (PRE-REDUCCIÓN)

Dimensiones: 161078 filas x 58 columnas

--- Esquema Actual ---
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = false)
 |-- FAMILY_SIZE: doubl

In [180]:

# 6. FEATURE ENGINEERING Y REDUCCIÓN FINAL

print("FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL\n ")

# --- A. CREACIÓN DE KPIs (Información de Negocio) ---
print("Generando KPIs estratégicos...")

# 1. KPI Gasto Total Tarjeta (Suma de los parciales)
df_master = df_master.withColumn(
    "KPI_TOTAL_SPEND",
    F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER")
)

# 2. KPI Ratio Endeudamiento (Deuda / Ingresos)
# Sumamos 1 al ingreso para evitar divisiones por cero
df_master = df_master.withColumn(
    "KPI_DEBT_RATIO",
    F.round(F.col("CREDICT_CARD_BALANCE") / (F.col("TOTAL_INCOME") + 1), 4)
)

# 3. KPI Grupo de Edad (Simplificación demográfica)
df_master = df_master.withColumn(
    "KPI_AGE_GROUP",
    F.when(F.col("AGE_IN_YEARS") < 30, "Joven")
     .when(F.col("AGE_IN_YEARS") < 50, "Adulto")
     .otherwise("Senior")
)

# 4. KPI Volatilidad de Préstamos (Max - Min)
df_master = df_master.withColumn(
    "KPI_LOAN_VOLATILITY",
    F.col("LOAN_CREDIT_GRANTED_MAX") - F.col("LOAN_CREDIT_GRANTED_MIN")
)

# 5. KPI Ratio de Aprobación (Lo que le dieron / Lo que pidió)
# Indica la confianza del banco en el cliente
df_master = df_master.withColumn(
    "KPI_APPROVAL_RATIO",
    F.round(F.col("LOAN_CREDIT_GRANTED_SUM") / (F.col("LOAN_APPLICATION_AMOUNT_SUM") + 1), 2)
)

# 6. KPI Tasa de Rechazo (Solicitudes denegadas / Total)
# Resume los estados conflictivos
df_master = df_master.withColumn(
    "KPI_DENIAL_RATE",
    F.round(F.col("NUM_STATUS_DENIED") / 
            (F.col("NUM_STATUS_AUTHORIZED") + F.col("NUM_STATUS_DENIED") + F.col("NUM_STATUS_ANNULLED") + 1), 2)
)

# --- B. REDUCCIÓN DE COLUMNAS (Limpieza de "Grasa") ---
print("Eliminando columnas redundantes e inútiles...")

cols_a_borrar = [
    # 1. Redundantes de Tarjeta (Sustituidas por KPI_TOTAL_SPEND)
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER", 
    "CREDIT_CARD_DRAWINGS", # Dato duplicado original
    
    # 2. Redundantes de Préstamos (Sustituidas por KPI_LOAN_VOLATILITY y APPROVAL)
    "LOAN_ANNUITY_PAYMENT_MAX", "LOAN_ANNUITY_PAYMENT_MIN",
    "LOAN_APPLICATION_AMOUNT_MAX", "LOAN_APPLICATION_AMOUNT_MIN",
    "LOAN_CREDIT_GRANTED_MAX", "LOAN_CREDIT_GRANTED_MIN",
    "LOAN_VARIABLE_RATE_MAX", "LOAN_VARIABLE_RATE_MIN",

    # 3. Basura Técnica y Duplicados
    "CONTRACT_ID",  # ID interno técnico (inútil para negocio)
    "CURRENCY"      # Columna duplicada por el Join (borramos ambas)
    ]

# Borrado seguro (solo las que existan)
cols_finales_borrar = [c for c in cols_a_borrar if c in df_master.columns]
df_master = df_master.drop(*cols_finales_borrar)

# --- C. DIMENSIONES FINALES ---

print("ESTADO FINAL OPTIMIZADO\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print(f"(Se han eliminado {len(cols_finales_borrar)} columnas redundantes y creado 6 KPIs estratégicos)")

# --- D. GUARDADO FINAL ---
ruta_final = OUTPUT_PATH + "Master_FinPlus.parquet"
df_master.write.mode("overwrite").parquet(ruta_final)
print(f"\nGuardado archivo en: {ruta_final}.")

# --- E. GUARDADO EN PARQUET ÚNICO ---

df_pandas = df_master.toPandas()

nombre_archivo = "Master_FinPlus_Final.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# Necesitas tener instalada la librería pyarrow o fastparquet (suele venir en Docker)
df_pandas.to_parquet(ruta_completa, index=False)

print(f"\nGuardado.")
print(f" Ruta: {ruta_completa}")

FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL
 
Generando KPIs estratégicos...
Eliminando columnas redundantes e inútiles...
ESTADO FINAL OPTIMIZADO

Dimensiones: 161078 filas x 49 columnas
(Se han eliminado 14 columnas redundantes y creado 6 KPIs estratégicos)

Guardado archivo en: /home/jovyan/work/data/curated/Master_FinPlus.parquet.

Guardado.
 Ruta: /home/jovyan/work/data/curated/Master_FinPlus_Final.parquet


In [181]:
def auditar_outliers(df, cols_numericas):
    
    print(f" VERIFICACIÓN DE OUTLIERS (Método IQR)")
    
    
    for col in cols_numericas:
        # 1. Calculamos Cuartiles (25% y 75%)
        quantiles = df.stat.approxQuantile(col, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        
        # 2. Definimos límites (Bigotes del Boxplot)
        limite_inf = q1 - 1.5 * iqr
        limite_sup = q3 + 1.5 * iqr
        
        # 3. Contamos cuántos se salen
        outliers = df.filter((F.col(col) < limite_inf) | (F.col(col) > limite_sup))
        num_outliers = outliers.count()
        
        if num_outliers > 0:
            print(f"\n Columna: {col}")
            print(f"   Rango Normal: [{limite_inf:.2f}  a  {limite_sup:.2f}]")
            print(f"   Outliers detectados: {num_outliers} filas")
            
            # Mostramos los valores más extremos para ver si son errores o VIPs
            print(f"   Ejemplos (Top Extremos):")
            outliers.select(col).orderBy(F.desc(col)).show(3)
        else:
            print(f"\n Columna: {col} -> Sin outliers estadísticos.")

# Definimos las columnas numéricas críticas para analizar
cols_analisis = ["TOTAL_INCOME", "AGE_IN_YEARS", "AMOUNT_PRODUCT", "CREDICT_CARD_BALANCE"]

# Ejecutamos
auditar_outliers(df_master, cols_analisis)

 VERIFICACIÓN DE OUTLIERS (Método IQR)

 Columna: TOTAL_INCOME
   Rango Normal: [-270.00  a  4050.00]
   Outliers detectados: 7244 filas
   Ejemplos (Top Extremos):
+------------+
|TOTAL_INCOME|
+------------+
|   1404000.0|
|   216001.08|
|    108000.0|
+------------+
only showing top 3 rows


 Columna: AGE_IN_YEARS -> Sin outliers estadísticos.

 Columna: AMOUNT_PRODUCT
   Rango Normal: [-6455.70  a  19399.50]
   Outliers detectados: 3395 filas
   Ejemplos (Top Extremos):
+--------------+
|AMOUNT_PRODUCT|
+--------------+
|     48486.195|
|     48486.195|
|     48486.195|
+--------------+
only showing top 3 rows


 Columna: CREDICT_CARD_BALANCE
   Rango Normal: [0.00  a  0.00]
   Outliers detectados: 16332 filas
   Ejemplos (Top Extremos):
+--------------------+
|CREDICT_CARD_BALANCE|
+--------------------+
|            14526.13|
|            12122.07|
|            11500.21|
+--------------------+
only showing top 3 rows

